In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install keybert
!pip install sklearn

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:

from transformers import T5ForConditionalGeneration, T5Tokenizer
from keybert import KeyBERT
import random

qg_model_name = "valhalla/t5-small-qg-hl"
tokenizer = T5Tokenizer.from_pretrained(qg_model_name)
qg_model = T5ForConditionalGeneration.from_pretrained(qg_model_name)

kw_model = KeyBERT()

def generate_question(context, answer):
    highlighted = context.replace(answer, f"<hl> {answer} <hl>", 1)
    input_text = f"generate question: {highlighted}"
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    output_ids = qg_model.generate(input_ids, max_length=64, num_beams=4, early_stopping=True)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

def get_distractors(context, correct_answer, num_distractors=3):
    keywords = kw_model.extract_keywords(context, keyphrase_ngram_range=(1, 2), stop_words='english', top_n=20)
    distractors = []

    for word, _ in keywords:
        if word.lower() != correct_answer.lower() and correct_answer.lower() not in word.lower():
            distractors.append(word)
        if len(distractors) == num_distractors:
            break

    fallback = ["Option A", "Option B", "Option C", "Option D"]
    while len(distractors) < num_distractors:
        option = random.choice(fallback)
        if option.lower() != correct_answer.lower():
            distractors.append(option)

    return distractors

def generate_mcq_from_context(context):
    # Extract potential answers using KeyBERT
    keywords = kw_model.extract_keywords(context, keyphrase_ngram_range=(1, 2), stop_words='english', top_n=5)

    if not keywords:
        print("⚠️ No keywords found to generate questions.")
        return

    print("\n📚 Generated MCQs:\n")

    for keyword, _ in keywords:
        answer = keyword
        try:
            question = generate_question(context, answer)
            distractors = get_distractors(context, answer)
            options = distractors + [answer]
            random.shuffle(options)

            correct_letter = chr(65 + options.index(answer))

            print(f"Q: {question}")
            for i, opt in enumerate(options):
                print(f"{chr(65 + i)}. {opt}")
            print(f"✅ Answer: {correct_letter}. {answer}\n")

        except Exception as e:
            print(f"❌ Failed for keyword '{answer}': {e}\n")

def main():
    print("📘 MCQ Generator")
    context = input("\n📝 Enter the context paragraph:\n> ")
    generate_mcq_from_context(context)

if __name__ == "__main__":
    main()


📘 MCQ Generator

📝 Enter the context paragraph:
> Node.js operates on a single thread, using non-blocking I/O calls, allowing it to support tens of thousands of concurrent connections without incurring the cost of thread context switching. It uses asynchronous programming. A common task for a web server can be to open a file on the server and return the content to the client but servers should not be used for simple tasks when you can get them done without the help of servers. 

📚 Generated MCQs:

Q: How does Node.js operate on a single thread?
A. uses asynchronous
B. client servers
C. web server
D. node js
✅ Answer: D. node js

Q: What type of programming does Node.js use?
A. uses asynchronous
B. web server
C. client servers
D. node js
✅ Answer: A. uses asynchronous

Q: How does Node.js operate on a single thread?
A. node js
B. uses asynchronous
C. web server
D. client servers
✅ Answer: D. client servers

Q: What is a common task for?
A. node js
B. client servers
C. uses asynchronous
